In [136]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import preprocessing

import string
import csv

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

from keras.models import Sequential, load_model, Model
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Flatten, MaxPool2D
from keras.optimizers import RMSprop

from sklearn.svm import SVC


In [137]:
csv_Train_File_Path = r'C:\Users\Sachin13390\Desktop\Python_Practice_Files\Kaggle_Titanic\all\train - Copy.csv'
csv_Test_File_Path = r'C:\Users\Sachin13390\Desktop\Python_Practice_Files\Kaggle_Titanic\all\test.csv'
# csv_Submission_File_Path = r'C:\Users\Sachin13390\Desktop\Kaggle-Emnist\My_Submission5.csv'

model_File_Path = r'all/Classifier.h5'

In [138]:
def Calculate_Total_Family_Member(df):
    
    Total_Family_Member = []
    
    for i in range (len(df)):
        Total_Family_Member.append(int(df.iloc[i]['SibSp']) + int(df.iloc[i]['Parch']))
        
    df['Total_Family_Member'] = Total_Family_Member
    
    return df


def Get_Average_Age_Per_Class(df, class_type):
    count = 0
    addition = 0.0
    
    for i in range (len(df)):
        
        if  (df.iloc[i]['Pclass'] == class_type ) and (str(type(df.iloc[i]['Age'])) != 'nan') and (float(df.iloc[i]['Age']) > 0.0):        
            addition = addition + float(df.iloc[i]['Age'])
            count = count + 1
            
    average = addition/count
    
    return average


def Missing_Age_Value(df_train, df_test):
    
    unique_class = df_train['Pclass'].unique()
    
    merged_df = frames = [df_train, df_test]
    merged_df = pd.concat(merged_df)
    
    for class_type in unique_class:
        
        average = Get_Average_Age_Per_Class(merged_df, class_type)
            
        for i in range (len(df_train)):
            if (str(df_train.iloc[i]['Pclass']) == str(class_type) ) and (str(df_train.iloc[i]['Age']) == 'nan') :
                df_train.at[i,'Age'] = average
    
    return df_train


def Remove_Unwanted_Column(df, columns_to_remove = []):
    
    if (df['Cabin'].isnull().any().any()) == True:
        df['Cabin'] = df['Cabin'].fillna('Missing')
            
    cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Missing']
    
    for i in range(len(df)):
        for cabin in cabin_list:
            if cabin in df.iloc[i]['Cabin']:
                df.at[i, 'Cabin'] = cabin

    encoder = LabelEncoder()
    df['Cabin']  = encoder.fit_transform(df['Cabin'] )
    
    if (len(columns_to_remove)==0):
        columns_to_remove = ['PassengerId', 'Name', 'Ticket', 'Fare' ]
    
    print(columns_to_remove)
        
    df = df.drop(columns_to_remove, axis=1)
    
    return df
   

    
def Label_Encode_Column(df):
    
    encoder = LabelEncoder()
    
    for i in range (df.shape[1]):
        
        if df.dtypes[i] == 'object':
            
            if (df[df.columns[i]].isnull().any().any()) == True:
                df[df.columns[i]] = df[df.columns[i]].fillna('Missing')
                df[df.columns[i]] = df[df.columns[i]].astype('category')
            
            df[df.columns[i]]  = encoder.fit_transform(df[df.columns[i]] )
    
    return df

def Category_Hot_Encode(df, columns_To_Hot_Encode=[]):
    
    if (len(columns_To_Hot_Encode)==0):
        Pclass = 1
        Sex = 2
        Embarked = 6
        
        columns_To_Hot_Encode = [Embarked, Sex, Pclass]
        
#     one_Hot_Encoder = OneHotEncoder(categorical_features=[1])
#     df = one_Hot_Encoder.fit_transform(df).toarray()
        
    for i in range (len(columns_To_Hot_Encode)):
        print(columns_To_Hot_Encode[i])
        one_Hot_Encoder = OneHotEncoder(categorical_features=[columns_To_Hot_Encode[i]])
        df = one_Hot_Encoder.fit_transform(df).toarray()
    
    df = pd.DataFrame(df)
    
    return df 



def Encoding_Columns(df, columns_to_Encode = []):
    
    if (len(columns_to_Encode)==0):
        columns_to_Encode = ['Pclass']
        
    df = pd.get_dummies(df)
    
    for column in columns_to_Encode:
        encoded_Column = pd.get_dummies(df[column], prefix = column)
        df[encoded_Column.columns] = encoded_Column
        df = df.drop(column, axis=1)
        
    return df   
    


In [139]:
def Data_Modification(df, df_test):
    
    df = Calculate_Total_Family_Member(df)
    
    df = Missing_Age_Value(df, df_test)
    Max_Age_Value = 80.0
    df['Age'] = df['Age']/Max_Age_Value
    
    df = Remove_Unwanted_Column(df)

    df = Encoding_Columns(df)

    return df

# ANN

In [216]:
df_test = pd.read_csv(csv_Test_File_Path)
df = pd.read_csv(csv_Train_File_Path)

df = Data_Modification(df, df_test)
df

['PassengerId', 'Name', 'Ticket', 'Fare']


,Survived,Age,SibSp,Parch,Cabin,Total_Family_Member,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,0,0.275000,1,0,7,1,0,1,0,0,1,0,0,1
1,1,0.475000,1,0,2,1,1,0,1,0,0,1,0,0
2,1,0.325000,0,0,7,0,1,0,0,0,1,0,0,1
3,1,0.437500,1,0,2,1,1,0,0,0,1,1,0,0
4,0,0.437500,0,0,7,0,0,1,0,0,1,0,0,1
5,0,0.310205,0,0,7,0,0,1,0,1,0,0,0,1
6,0,0.675000,0,0,4,0,0,1,0,0,1,1,0,0
7,0,0.025000,3,1,7,4,0,1,0,0,1,0,0,1
8,1,0.337500,0,2,7,2,1,0,0,0,1,0,0,1
9,1,0.175000,1,0,7,1,1,0,1,0,0,0,1,0


In [141]:
X = df.drop(df.columns[0],axis = 1)
y = df[df.columns[0]]

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [142]:
classifier = Sequential()

classifier.add(Dense(units = X_train.shape[1], kernel_initializer = 'uniform', activation = 'relu',input_dim = X_train.shape[1]))
classifier.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu' ))
classifier.add(Dense(units = X_train.shape[1], kernel_initializer = 'uniform', activation = 'relu' ))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid' ))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [143]:
classifier.fit(X_train, Y_train,
               batch_size=16, epochs=100,
               verbose=2,
               validation_data=(X_test, Y_test))

Train on 801 samples, validate on 90 samples
Epoch 1/100
 - 0s - loss: 0.6782 - acc: 0.6142 - val_loss: 0.6338 - val_acc: 0.6000
Epoch 2/100
 - 0s - loss: 0.6082 - acc: 0.6392 - val_loss: 0.5518 - val_acc: 0.7333
Epoch 3/100
 - 0s - loss: 0.5335 - acc: 0.7690 - val_loss: 0.4533 - val_acc: 0.8222
Epoch 4/100
 - 0s - loss: 0.4822 - acc: 0.8052 - val_loss: 0.4227 - val_acc: 0.8111
Epoch 5/100
 - 0s - loss: 0.4520 - acc: 0.8040 - val_loss: 0.4256 - val_acc: 0.8111
Epoch 6/100
 - 0s - loss: 0.4429 - acc: 0.8065 - val_loss: 0.4194 - val_acc: 0.8222
Epoch 7/100
 - 0s - loss: 0.4397 - acc: 0.7990 - val_loss: 0.4155 - val_acc: 0.8333
Epoch 8/100
 - 0s - loss: 0.4389 - acc: 0.8065 - val_loss: 0.4229 - val_acc: 0.8333
Epoch 9/100
 - 0s - loss: 0.4415 - acc: 0.7928 - val_loss: 0.4184 - val_acc: 0.8444
Epoch 10/100
 - 0s - loss: 0.4298 - acc: 0.8240 - val_loss: 0.4315 - val_acc: 0.8333
Epoch 11/100
 - 0s - loss: 0.4314 - acc: 0.8090 - val_loss: 0.4165 - val_acc: 0.8444
Epoch 12/100
 - 0s - loss: 0.

Epoch 97/100
 - 0s - loss: 0.3904 - acc: 0.8340 - val_loss: 0.4355 - val_acc: 0.8333
Epoch 98/100
 - 0s - loss: 0.3896 - acc: 0.8340 - val_loss: 0.4211 - val_acc: 0.8222
Epoch 99/100
 - 0s - loss: 0.3959 - acc: 0.8302 - val_loss: 0.4611 - val_acc: 0.8333
Epoch 100/100
 - 0s - loss: 0.3946 - acc: 0.8390 - val_loss: 0.4395 - val_acc: 0.8222


In [ ]:
# classifier.save(model_File_Path)

In [112]:
csv_Submission_File_Path = 'all\My_Submission.csv'

In [145]:
df_test = pd.read_csv(csv_Test_File_Path)
df = pd.read_csv(csv_Train_File_Path)

df_test = Data_Modification(df_test, df)

['PassengerId', 'Name', 'Ticket', 'Fare']


In [146]:
threshold = 0.65

columns=['PassengerId', 'Survived']
output = []
start_value = 892

y_pred = classifier.predict(df_test)
y_pred = (y_pred > threshold)

count_Positive = 0

for i in range(len(y_pred)):
    
    value = 0
    
    if ((y_pred[i][0]) == True):
        value = 1
        count_Positive = count_Positive + 1
    
    row = [start_value, value]
    output.append(row)
    
    start_value = start_value + 1
    
ouput_df = pd.DataFrame(output, columns=columns)

count_Positive

125

In [ ]:
ouput_df.to_csv(csv_Submission_File_Path, index=False)

# SVM

In [147]:
df_test = pd.read_csv(csv_Test_File_Path)
df = pd.read_csv(csv_Train_File_Path)

df = Data_Modification(df, df_test)

['PassengerId', 'Name', 'Ticket', 'Fare']


In [148]:
X = df.drop(df.columns[0],axis = 1).values
y = df[df.columns[0]].values

labelencoder_y_1 = LabelEncoder()
y = labelencoder_y_1.fit_transform(y)

In [149]:
from sklearn.svm import SVC

SVM_classifier = SVC(random_state = 0)

SVM_classifier.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [150]:
csv_Submission_File_Path = 'all\My_Submission.csv'

In [151]:
df_test = pd.read_csv(csv_Test_File_Path)
df = pd.read_csv(csv_Train_File_Path)

df_test = Data_Modification(df_test, df)

['PassengerId', 'Name', 'Ticket', 'Fare']


In [152]:
y_pred = SVM_classifier.predict(df_test)

In [153]:
count_Positive = 0

columns=['PassengerId', 'Survived']
output = []
start_value = 892

for i in range(len(y_pred)):
    
    if(y_pred[i] == 1):
        count_Positive = count_Positive + 1
    
#     print (y_pred[i])
    
    row = [start_value, y_pred[i]]
    output.append(row)
    
    start_value = start_value + 1
    
ouput_df = pd.DataFrame(output, columns=columns)

count_Positive

148

In [154]:
ouput_df.to_csv(csv_Submission_File_Path, index=False)

# Random Forest

In [208]:
df_test = pd.read_csv(csv_Test_File_Path)
df = pd.read_csv(csv_Train_File_Path)

df = Data_Modification(df, df_test)

['PassengerId', 'Name', 'Ticket', 'Fare']


In [209]:
X = df.drop(df.columns[0],axis = 1).values
y = df[df.columns[0]].values

labelencoder_y_1 = LabelEncoder()
y = labelencoder_y_1.fit_transform(y)

In [210]:
from sklearn.ensemble import RandomForestClassifier

In [211]:
random_Forest_classifier = RandomForestClassifier( n_estimators = 100, criterion = 'entropy' ,random_state = 0)

random_Forest_classifier.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [212]:
df_test = pd.read_csv(csv_Test_File_Path)
df = pd.read_csv(csv_Train_File_Path)

df_test = Data_Modification(df_test, df)

['PassengerId', 'Name', 'Ticket', 'Fare']


In [213]:
y_pred = random_Forest_classifier.predict(df_test)
# y_pred

In [214]:
count_Positive = 0

columns=['PassengerId', 'Survived']
output = []
start_value = 892

for i in range(len(y_pred)):
    
    if(y_pred[i] == 1):
        count_Positive = count_Positive + 1
    
#     print (y_pred[i])
    
    row = [start_value, y_pred[i]]
    output.append(row)
    
    start_value = start_value + 1
    
ouput_df = pd.DataFrame(output, columns=columns)

count_Positive

163

In [215]:
ouput_df.to_csv(csv_Submission_File_Path, index=False)